In [1]:
import os
import datetime
import json
import pandas as pd
import numpy as np
import betfairlightweight as bfl

with open('secrets.json') as f:
    secrets = json.loads(f.read())

trading = bfl.APIClient(secrets['username'], secrets['password'], app_key=secrets['app_key'], cert_files=['./certs/client-2048.crt', './certs/client-2048.key'])
trading.login()


<LoginResource>

In [2]:
event_types = trading.betting.list_event_types()
sport_ids = pd.DataFrame({
    'Sport': [event_type_object.event_type.name for event_type_object in event_types],
    'ID': [event_type_object.event_type.id for event_type_object in event_types]
}).set_index('Sport').sort_index()

sport_ids

,ID
Sport,
American Football,6423
Athletics,3988
Australian Rules,61420
Baseball,7511
Basketball,7522
Boxing,6
Cricket,4
Current Affairs,27388198
Cycling,11


In [8]:
# Define a market filter
event_filter = bfl.filters.market_filter(
    event_type_ids=[7],
    market_countries=['GB'],
    market_start_time={
        'from': datetime.datetime.utcnow().strftime("%Y-%m-%dT%TZ"),
        'to': (datetime.datetime.utcnow() + datetime.timedelta(days=1)).strftime("%Y-%m-%dT%TZ"),
    }
)

# Print the filter
event_filter

{'eventTypeIds': [7],
 'marketCountries': ['GB'],
 'marketStartTime': {'from': '2019-11-03T09:57:07Z',
  'to': '2019-11-04T09:57:07Z'}}

In [9]:
# Get a list of all thoroughbred events as objects
gb_events = trading.betting.list_events(filter=event_filter)

# Create a DataFrame with all the events by iterating over each event object
gb_events_today = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in gb_events],
    'Event ID': [event_object.event.id for event_object in gb_events],
    'Event Venue': [event_object.event.venue for event_object in gb_events],
    'Country Code': [event_object.event.country_code for event_object in gb_events],
    'Time Zone': [event_object.event.time_zone for event_object in gb_events],
    'Open Date': [event_object.event.open_date for event_object in gb_events],
    'Market Count': [event_object.market_count for event_object in gb_events]
})

gb_events_today

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Carl (RFC) 3rd Nov,29564657,Carlisle,GB,Europe/London,2019-11-03 12:45:00,3
1,Carl (F/C) 3rd Nov,29564677,Carlisle,GB,Europe/London,2019-11-03 12:45:00,1
2,Hunt (RFC) 3rd Nov,29564644,Huntingdon,GB,Europe/London,2019-11-03 12:05:00,7
3,Hunt 3rd Nov,29562858,Huntingdon,GB,Europe/London,2019-11-03 12:05:00,40
4,Carl 3rd Nov,29561034,Carlisle,GB,Europe/London,2019-11-03 12:45:00,31
